In [2]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.preprocessing import StandardScaler
from utils import *
from torch.utils.tensorboard import SummaryWriter
import datetime
import time
#from model import InceptionTimePlus
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_curve, roc_auc_score,precision_recall_fscore_support,f1_score,accuracy_score,precision_score,recall_score,balanced_accuracy_score

In [3]:
data = pd.read_csv('data_multiClass.csv')
data = data.drop('Unnamed: 0', axis=1)
data

,640,641,642,643,644,645,646,647,648,649,...,1042,1043,1044,1045,1046,1047,1048,1049,1050,category
0,0.126331,0.126273,0.126215,0.126215,0.126040,0.125808,0.125634,0.125692,0.125866,0.125808,...,0.149905,0.149783,0.149538,0.149231,0.148803,0.148253,0.147764,0.147642,0.147947,3
1,0.142547,0.142366,0.142245,0.142185,0.142065,0.141824,0.141403,0.141042,0.140742,0.140561,...,0.177309,0.177113,0.176330,0.175094,0.174055,0.173472,0.173277,0.172825,0.171920,3
2,0.067932,0.068034,0.068135,0.068186,0.068186,0.068288,0.068390,0.068593,0.068898,0.069357,...,0.127844,0.127494,0.127669,0.127494,0.126040,0.125228,0.125344,0.124707,0.123955,3
3,0.033483,0.033342,0.033248,0.033201,0.033108,0.032920,0.032733,0.032592,0.032546,0.032686,...,0.085551,0.086557,0.086557,0.085287,0.083757,0.083073,0.083336,0.083599,0.083651,3
4,0.082862,0.082862,0.082967,0.083073,0.083020,0.082915,0.082810,0.082862,0.083125,0.083493,...,0.142969,0.142366,0.142607,0.142547,0.141102,0.140381,0.140621,0.139542,0.138764,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,0.187421,0.186953,0.186419,0.185952,0.185553,0.185153,0.184821,0.184489,0.184090,0.183825,...,0.093557,0.093396,0.093503,0.093611,0.093611,0.092965,0.091890,0.091461,0.091301,0
2562,0.223226,0.222863,0.222501,0.222138,0.221776,0.221415,0.221126,0.220764,0.220404,0.220187,...,0.138048,0.138346,0.138585,0.137690,0.135608,0.134896,0.135726,0.135548,0.134837,0
2563,0.216168,0.215954,0.215597,0.215169,0.214670,0.214243,0.213888,0.213604,0.213391,0.213249,...,0.137988,0.136499,0.136083,0.135964,0.134955,0.134541,0.135015,0.135074,0.134363,0
2564,0.246340,0.245958,0.245422,0.244888,0.244430,0.244201,0.244049,0.243744,0.243288,0.242832,...,0.173277,0.171340,0.171405,0.171920,0.170632,0.169925,0.170632,0.170761,0.169861,0


In [4]:
# 特征和标签
X = data.drop('category', axis=1).values
y = data['category'].values
print(X.shape)
print(y.shape)

(2566, 411)
(2566,)


In [5]:
#标准化特征
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(1/2), random_state=42)

# 将数据转换为卷积神经网络输入格式 (samples, timesteps, features)
# 将数据转换为卷积神经网络输入格式 (samples, channels, sequence_length)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1539, 1, 411)
(1539,)
(513, 1, 411)
(513,)
(514, 1, 411)
(514,)


In [6]:
# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor  = F.one_hot(torch.tensor(y_train)).float()
y_val_tensor = F.one_hot(torch.tensor(y_val)).float()
y_test_tensor = F.one_hot(torch.tensor(y_test)).float()


# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
X_train_tensor.shape

torch.Size([1539, 1, 411])

In [7]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            # m.bias.data.zero_()

        elif isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
            
# 定义卷积神经网络模型
class CNN1D(nn.Module):
    def __init__(self, input_length, num_class=4):
        super(CNN1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=17)
        self.bn2 = nn.BatchNorm1d(64)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=47)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self._get_conv_output_size(input_length), 128)
        self.fc2 = nn.Linear(128, num_class)

        initialize_weights(self)
        
    def _get_conv_output_size(self, input_length):
        size = input_length
        size = (size - 4) // 2  # conv1 and pool1
        size = (size - 16) // 2  # conv2 and pool2
        size = (size - 46) // 2  # conv3 and pool3
        return size * 128

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
CNN1D(411)

CNN1D(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(17,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(47,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2944, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [7]:
def train(model, criterion, optimizer, scheduler, n_epochs=5):
    best = 0
    train_step = 0
    losses = []

    # 定义自定义日志目录名称
    log_dir_name = "V1_6_multiClass"   # "V3_multiClass" 'V3_2_binary'
    log_dir = f"runs/{log_dir_name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
    
    # 初始化TensorBoard
    writer = SummaryWriter(log_dir=log_dir)
    print(f"tensorboard --logdir={log_dir}")

    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        train_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, targets = data
            
            inputs = inputs.cuda()
            targets = targets.cuda()
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, targets)
            loss.backward()

            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()

            train_loss += loss.item()
            writer.add_scalar("train loss", loss.item(), train_step)
            train_step += 1
        
        epoch_duration = time.time() - since
        epoch_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, duration: {epoch_duration:.2f}, loss: {epoch_loss:.6f}")

        losses.append(epoch_loss)

        # 验证模型
        model.eval()
        val_acc = val(model, val_loader)
        writer.add_scalar("val acc", val_acc, epoch + 1)
        if best < val_acc:
            best = val_acc
            torch.save(model, "weights/V1_6_multiClass.pth")

        model.train()
        #scheduler.step() # test_acc
    
    print('Finished Training')
    return model, losses

In [8]:
def val(model, val_dataloader):
    val_labels = []
    val_predictions = []
    with torch.no_grad():
        for i, (inputs, label) in enumerate(val_dataloader, 0):
            # images = images.to(device).half() # uncomment for half precision model
            inputs = inputs.cuda()
            label = label.cuda()
            outputs = model(inputs)

            val_labels.extend([label.cpu().numpy()])
            val_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])
    
    val_labels = np.vstack(val_labels)
    val_predictions = np.vstack(val_predictions)

    val_predictions_prob = np.exp(val_predictions)/np.sum(np.exp(val_predictions),axis=1,keepdims=True)
    
    
    val_predictions = np.argmax(val_predictions,axis=1)
    val_labels = np.argmax(val_labels,axis=1)
    avg_score = accuracy_score(val_labels,val_predictions)
    print('Accuracy of the network on the test dataset: {}'.format(avg_score))
    return avg_score

In [19]:
# 定义OneCycleLR学习率调整策略
learning_rate = 5e-5
num_classes = 2
epochs = 100

# model_ft = WaveLength(c_in=X.shape[1], c_out=6, nf=[47, 47, 47, 47])
model_ft = CNN1D(411, 4).cuda()
criterion = nn.CrossEntropyLoss() # nn.BCEWithLogitsLoss()

#lrscheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, epochs=epochs, steps_per_epoch=len(train_dataloader))
# lrscheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20, eta_min=1e-9)


optimizer = optim.Adam(model_ft.parameters(), lr=learning_rate, weight_decay=learning_rate*0.1)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=17, verbose=True)
# parms_1x = [value for name, value in model.named_parameters()
#             if name not in ["fc.weight", "fc.bias"]]
# parms_10x = [value for name, value in model.named_parameters()
#             if name in ["fc.weight", "fc.bias"]]
# optimizer = optim.Adam([{"params": parms_1x},
#                         {"params": parms_10x, 'lr': learning_rate * 10}], lr=learning_rate)

total_params = sum(p.numel() for p in model_ft.parameters())
print(f'Total parameters: {total_params}')  # 798406 参数大概合理
model_ft

Total parameters: 798148


/root/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


CNN1D(
  (conv1): Conv1d(1, 32, kernel_size=(5,), stride=(1,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv1d(32, 64, kernel_size=(17,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv1d(64, 128, kernel_size=(47,), stride=(1,))
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2944, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [20]:
model_ft, training_losses = train(model_ft, criterion, optimizer, lrscheduler, n_epochs=epochs)

tensorboard --logdir=runs/V1_6_multiClass_20240612-025226
Epoch 1, duration: 0.32, loss: 1.374939
Accuracy of the network on the test dataset: 0.26900584795321636
Epoch 2, duration: 0.32, loss: 1.265772
Accuracy of the network on the test dataset: 0.31773879142300193
Epoch 3, duration: 0.32, loss: 1.161101
Accuracy of the network on the test dataset: 0.3684210526315789
Epoch 4, duration: 0.32, loss: 1.084077
Accuracy of the network on the test dataset: 0.43664717348927873
Epoch 5, duration: 0.32, loss: 0.986192
Accuracy of the network on the test dataset: 0.4600389863547758
Epoch 6, duration: 0.32, loss: 0.965795
Accuracy of the network on the test dataset: 0.5048732943469786
Epoch 7, duration: 0.32, loss: 0.942783
Accuracy of the network on the test dataset: 0.5458089668615984
Epoch 8, duration: 0.32, loss: 0.861615
Accuracy of the network on the test dataset: 0.5886939571150097
Epoch 9, duration: 0.32, loss: 0.839662
Accuracy of the network on the test dataset: 0.6140350877192983
Epo

In [10]:
# model = WaveLength(c_in=411, c_out=6, nf=[47, 128, 128, 47])
model = torch.load('weights/V1_6_multiClass.pth')
model.eval()  # 切换模型到评估模式
print("Model loaded successfully.")

Model loaded successfully.


In [11]:
test_labels = []
test_predictions = []
with torch.no_grad():
    for i, (inputs, label) in enumerate(test_loader, 0):
        # images = images.to(device).half() # uncomment for half precision model
        inputs = inputs.cuda()
        label = label.cuda()
        outputs = model(inputs)

        test_labels.extend([label.cpu().numpy()])
        test_predictions.extend([torch.sigmoid(outputs).cpu().numpy()])

test_labels = np.vstack(test_labels)
test_predictions = np.vstack(test_predictions)

test_predictions_prob = np.exp(test_predictions)/np.sum(np.exp(test_predictions),axis=1,keepdims=True)


test_predictions = np.argmax(test_predictions,axis=1)
test_labels = np.argmax(test_labels,axis=1)

avg_score = accuracy_score(test_labels, test_predictions)
p_marco = precision_score(test_labels,test_predictions,average='macro')
r_marco = recall_score(test_labels,test_predictions,average='macro')
f1_marco = f1_score(test_labels,test_predictions,average='macro')
print("Average accuracy: {:.4f}".format(avg_score))
print("f1_marco: {:.4f}".format(f1_marco))
print("p_marco: {:.4f}".format(p_marco))
print("r_marco: {:.4f}".format(r_marco))

Average accuracy: 0.9436
f1_marco: 0.9439
p_marco: 0.9432
r_marco: 0.9452


In [23]:
report = classification_report(test_labels, test_predictions)
print(report)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       138
           1       0.94      0.97      0.96       105
           2       0.93      0.88      0.90       138
           3       0.92      0.94      0.93       133

    accuracy                           0.94       514
   macro avg       0.94      0.95      0.94       514
weighted avg       0.94      0.94      0.94       514



In [14]:
test_predictions

array([1, 3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 0, 3, 1, 2, 2, 0, 2, 0, 2, 2, 0,
       2, 1, 2, 3, 0, 2, 3, 1, 0, 2, 0, 3, 2, 0, 1, 1, 2, 3, 0, 3, 2, 1,
       0, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 2, 2, 1, 1, 3, 1, 2, 3, 1, 2,
       1, 2, 2, 0, 2, 0, 0, 3, 0, 2, 2, 0, 2, 1, 3, 2, 3, 0, 2, 0, 3, 1,
       1, 3, 3, 3, 3, 1, 3, 2, 1, 1, 2, 2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 3,
       1, 0, 1, 0, 2, 0, 1, 0, 0, 2, 3, 2, 3, 0, 2, 3, 0, 2, 1, 0, 2, 3,
       1, 0, 3, 3, 2, 2, 0, 1, 2, 2, 3, 2, 3, 3, 2, 2, 0, 3, 3, 2, 0, 2,
       3, 3, 1, 3, 2, 0, 1, 3, 2, 3, 0, 2, 0, 3, 0, 1, 1, 0, 0, 2, 1, 0,
       2, 3, 1, 2, 0, 0, 2, 0, 3, 0, 3, 3, 0, 0, 2, 1, 0, 1, 3, 3, 2, 0,
       0, 2, 0, 3, 3, 2, 1, 0, 1, 0, 1, 3, 2, 2, 1, 1, 0, 0, 1, 0, 1, 1,
       2, 1, 0, 2, 3, 0, 1, 3, 1, 3, 3, 0, 0, 0, 3, 0, 0, 1, 0, 0, 3, 2,
       0, 2, 2, 1, 2, 0, 3, 1, 0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 2, 2, 3, 0,
       2, 0, 2, 0, 3, 1, 3, 3, 1, 0, 3, 3, 3, 2, 0, 3, 3, 1, 2, 2, 0, 0,
       3, 3, 1, 3, 1, 0, 2, 0, 0, 3, 2, 0, 3, 3, 0,

In [15]:
test_labels

array([1, 3, 3, 1, 2, 1, 3, 1, 3, 2, 2, 0, 3, 1, 2, 2, 0, 2, 0, 2, 2, 0,
       2, 1, 1, 3, 0, 2, 3, 1, 0, 2, 0, 2, 2, 0, 1, 1, 2, 3, 0, 3, 2, 1,
       0, 2, 3, 3, 3, 0, 0, 2, 2, 1, 2, 1, 2, 2, 1, 1, 3, 1, 2, 3, 3, 2,
       1, 2, 2, 0, 2, 0, 0, 3, 0, 2, 2, 0, 2, 1, 3, 2, 3, 0, 2, 0, 3, 3,
       1, 3, 3, 3, 3, 1, 3, 2, 1, 1, 2, 2, 0, 1, 1, 1, 0, 1, 2, 2, 2, 3,
       1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 3, 2, 3, 0, 2, 3, 0, 2, 1, 0, 2, 3,
       1, 0, 3, 3, 3, 2, 0, 1, 3, 2, 3, 2, 3, 3, 2, 2, 0, 2, 3, 2, 0, 2,
       3, 3, 1, 3, 2, 1, 1, 2, 2, 3, 0, 3, 0, 3, 0, 1, 1, 0, 0, 2, 1, 0,
       2, 3, 1, 2, 0, 0, 2, 0, 3, 0, 2, 3, 0, 0, 2, 1, 0, 1, 3, 3, 2, 0,
       0, 2, 0, 3, 3, 2, 2, 0, 1, 0, 1, 3, 2, 2, 1, 1, 1, 0, 1, 0, 1, 1,
       2, 1, 0, 2, 3, 0, 1, 3, 1, 3, 3, 0, 0, 0, 2, 0, 0, 1, 0, 0, 3, 2,
       0, 2, 2, 1, 2, 0, 3, 1, 0, 0, 2, 1, 0, 2, 0, 0, 1, 1, 2, 2, 3, 0,
       3, 1, 3, 0, 3, 2, 3, 3, 1, 0, 3, 3, 2, 2, 0, 3, 2, 1, 2, 2, 0, 0,
       3, 3, 1, 3, 1, 0, 2, 0, 0, 3, 2, 0, 3, 3, 0,

In [13]:
# [1 1 2 2 3 3 4 4 5 5 0 0]
# [1 0 2 2 3 5 5 5 5 5 0 0]